In [2]:
import torch
from torch import nn
import os
import cv2
import matplotlib.pyplot as plt
import  numpy as np
from torch.utils.data import DataLoader
from torchvision import transforms, models
# import gdown

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

# face_localization_model = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

cuda:0


Upload your kaggle API key to download dataset

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle

!kaggle datasets download -d stoicstatic/face-recognition-dataset

if not os.path.exists("Extracted Faces"):
    !unzip -q face-recognition-dataset.zip "Extracted Faces/*"

Dataset URL: https://www.kaggle.com/datasets/stoicstatic/face-recognition-dataset
License(s): CC0-1.0
face-recognition-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


### Data preprocessing

In [4]:
data = torch.load("face_data2.pth")

train_dataset = data["train"]
test_dataset = data["test"]

/home/ubuntu/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [5]:
batch_size = 8
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

### Model

In [6]:
# model creation
additional_layer = nn.Linear(2048, 256)

resnet = models.resnet50(weights="DEFAULT")
encoder_layers = list(resnet.children())[:-1]
encoder_model = torch.nn.Sequential(*encoder_layers)

for param in encoder_model.parameters():
    param.requires_grad = False

model = nn.Sequential(encoder_model, nn.Flatten(), nn.Linear(2048, 256)).to(device)

In [7]:
def train(model, lr, n_epochs, dataloader):
    optimizer = torch.optim.Adam(model.parameters(), lr)

    for i in range(n_epochs):
        epoch_loss = 0
        print(accuracy(model, dataloader)[0], accuracy(model, test_dataloader)[0])
        
        for input, label in dataloader:
            input = input.to(device)
            label = label.to(device)
            combined_input = torch.cat((input[:, 0:3], input[:, 3:]), dim=0)
            output = model(combined_input)
            val1, val2 = torch.split(output, output.shape[0] // 2)

            distance = nn.functional.pairwise_distance(val1, val2, keepdim=True)
            # print(val1, val2)
            # print(distance)
            # break
            margin = 2
            loss = torch.mean((label) * distance**2 + (1 - label) * torch.clamp(margin - distance, min=0.0)**2)

            epoch_loss += loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print("loss: ", epoch_loss.item() / len(dataloader))

    # print(accuracy(model, dataloader)[0], accuracy(model, test_dataloader)[0])
        print(accuracy(model, dataloader)[1])

# model = SiameseCNN().to(device)

def accuracy(model, dataloader):
    model.eval()
    confusion_matrix = np.zeros((2, 2))


    for input, label in dataloader:
        input = input.to(device)
        label = label.to(device)
        combined_input = torch.cat((input[:, 0:3], input[:, 3:]), dim=0)
        output = model(combined_input)
        val1, val2 = torch.split(output, output.shape[0] // 2)

        distance = nn.functional.pairwise_distance(val1, val2, keepdim=True)
        margin = 2
        pred = (distance < margin).int()

        confusion_matrix[0][0] += torch.sum((pred == 1) & (label == 1)).item()
        confusion_matrix[0][1] += torch.sum((pred == 1) & (label == 0)).item()
        confusion_matrix[1][0] += torch.sum((pred == 0) & (label == 1)).item()
        confusion_matrix[1][1] += torch.sum((pred == 0) & (label == 0)).item()

        correct_ratio = np.trace(confusion_matrix) / np.sum(confusion_matrix)

    return correct_ratio, confusion_matrix / np.sum(confusion_matrix)

In [8]:
train(model, .001, 10, train_dataloader)
torch.save(model, "face_recognition_model.pth")

0.5165 0.5204081632653061
loss:  1.23926123046875
[[0.4865 0.4845]
 [0.0135 0.0155]]
0.50125 0.5
loss:  1.211468994140625
[[0.47575 0.47925]
 [0.02425 0.02075]]
0.49975 0.5025510204081632
loss:  1.382403076171875
[[0.47175 0.47225]
 [0.02825 0.02775]]
0.502 0.4872448979591837
loss:  1.2913807373046875
[[0.466125 0.470875]
 [0.033875 0.029125]]
0.50475 0.5
loss:  1.206465576171875
[[0.47675 0.47425]
 [0.02325 0.02575]]
0.5015 0.5051020408163265
loss:  1.242101806640625
[[0.4775 0.4775]
 [0.0225 0.0225]]
0.5 0.5
loss:  1.21904248046875
[[0.468875 0.464125]
 [0.031125 0.035875]]
0.50075 0.49744897959183676
loss:  1.5067196044921876
[[0.4765 0.4755]
 [0.0235 0.0245]]
0.49675 0.5076530612244898
loss:  1.4090745849609374
[[0.462625 0.463375]
 [0.037375 0.036625]]
0.501 0.5051020408163265
loss:  1.386261962890625
[[0.459875 0.463125]
 [0.040125 0.036875]]


In [9]:
# model = torch.load("face_recognition_model3.pth", map_location=device)
# model.eval()

print(accuracy(model, test_dataloader))
# print(accuracy(model, train_dataloader))

(0.5, array([[0.5, 0.5],
       [0. , 0. ]]))


In [10]:
def testImgPreprocess(img_path):
    img = cv2.imread(img_path,  cv2.IMREAD_GRAYSCALE)
    cordinates = face_localization_model.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5)
    print(cordinates)
    x, y, w, h = cordinates[0]
    img = img[x:x+w, y:y+h]

    img = img / 255
    img = cv2.resize(img, (128, 128))

    plt.imshow(img)
    plt.show()

    return img

img1 = testImgPreprocess("test1.jpg")
img2 = testImgPreprocess("test2.jpg")
img3 = testImgPreprocess("test3.jpg")

accuracy([[img2, img3]], [0])

dummy_input = torch.rand((1, 1, 128, 128)).to(device)
torch.onnx.export(model, dummy_input, "face_detection.onnx")

[ WARN:0@70.026] global loadsave.cpp:248 findDecoder imread_('test1.jpg'): can't open/read file: check file path/integrity


NameError: name 'face_localization_model' is not defined